This notebook will copy the versions of common dependencies from one `Directory.Packages.props` to another. The target `Directory.Packages.props` is the one found in the current directory.

In [2]:
#!set --name sourceFilePath --value @input:{"prompt":"Pick the source Directory.Packages.props file","type":"file","saveAs":"VisualStudio.Conversations Directory.Packages.props"}

using System;
using System.Collections.Generic;
using System.Xml.Linq;
using Microsoft.DotNet.Interactive;
using System.IO;

var targetFilePath = Path.Combine(Directory.GetCurrentDirectory(),  "Directory.Packages.props");

Using previously saved value for `VisualStudio.Conversations Directory.Packages.props`.

> 💡 To remove this value from your [SecretStore](https://learn.microsoft.com/en-us/powershell/module/microsoft.powershell.secretstore/?view=ps-modules), run the following command in a PowerShell cell:
> 
> ```powershell
>     Remove-Secret -Name "VisualStudio.Conversations Directory.Packages.props" -Vault DotnetInteractive
> ```

> 📝 For more information, see [SecretManagement](https://learn.microsoft.com/en-us/powershell/utility-modules/secretmanagement/overview?view=ps-modules).

In [3]:
using System.Xml.Linq;

// Load XML files
XDocument sourceDoc = XDocument.Load(sourceFilePath);
XDocument targetDoc = XDocument.Load(targetFilePath);

// Extract all PackageVersion elements from source and create dictionary
Dictionary<string, string> packageVersions = new Dictionary<string, string>();

foreach (var packageVersion in sourceDoc.Descendants("PackageVersion").Where(pv => !pv.Parent.Attributes().Any()))
{
    string include = packageVersion.Attribute("Include")?.Value;
    string version = packageVersion.Attribute("Version")?.Value;
    
    if (!string.IsNullOrEmpty(include) && !string.IsNullOrEmpty(version))
    {
        packageVersions[include] = version;
    }
}

// Update target document
bool changes = false;
foreach (var packageVersion in targetDoc.Descendants("PackageVersion"))
{
    string include = packageVersion.Attribute("Include")?.Value;
    
    if (!string.IsNullOrEmpty(include) && packageVersions.TryGetValue(include, out string sourceVersion))
    {
        string targetVersion = packageVersion.Attribute("Version")?.Value;
        
        if (targetVersion != sourceVersion)
        {
            packageVersion.Attribute("Version").Value = sourceVersion;
            changes = true;
            Console.WriteLine($"Updated {include}: {targetVersion} -> {sourceVersion}");
        }
    }
}

// Save changes if any were made
if (changes)
{
    targetDoc.Save(targetFilePath);
    Console.WriteLine("Changes saved successfully.");
}
else
{
    Console.WriteLine("No changes were needed.");
}

Updated Markdig.Signed: 0.33.0.0 -> 0.30.0.0
Updated System.Drawing.Common: 9.0.1 -> 9.0.3
Updated xunit: 2.7.0 -> 2.9.0
Changes saved successfully.
